In [1]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
import dash_core_components as dcc
import numpy as np
from collections import Counter
from tqdm.auto import tqdm

In [2]:
from utils import EndoMaterial
from datetime import datetime as dt
from datetime import date

In [3]:
from fpdf import FPDF, TitleStyle

In [4]:
path_mat_doc = "data/Materialdokumentation.xlsx"
path_imd = "data/IMD_Endoskopie.xlsx"
path_lookup_dgvs_keys = "data/DGVS-Leistungskatalog-Leistungsgruppen-fuer-OPS-Version-2021.csv"
em = EndoMaterial(path_imd, path_mat_doc)

Dropped 31 rows at 'documentation_status'
Dropped 1 rows at 'intervention_type'
mat data contains 623 ids which are not found in imd data, removing them
imd data contains 77 ids which are not found in imd data, removing them
Dropped 1 rows at 'ID'
{'imd_init': 2862, 'mat_init': 1830, 'imd_after_processing': 2479, 'mat_after_processing': 1654}


In [5]:
_ = pd.read_csv(path_lookup_dgvs_keys, sep = ";", encoding = "latin1")
dgvs_lookup = {}
for index, row in _.iterrows():
    dgvs_lookup[row["Code"]] = row["Leistungsgruppe"]

In [5]:
# em.mat.head()

In [10]:
summary = em.generate_summary()

In [50]:
author = "T. J. Lux"
title_style_0 = TitleStyle("Times", "B", 16)
title_style_1 = TitleStyle("Times", "B", 14)
title_style_2 = TitleStyle("Times", "B", 14)
title_style_3 = TitleStyle("Times", "B", 14)
date_format = "%d.%m.%Y"

class PDF(FPDF):
    def header(self):
        # Logo
        self.image('data/logo.png', 8, 8, 15)
        # helvetica bold 15
        self.set_font('Times', '', 10)
        # Move to the right
        self.cell(20)
        # Title
        self.cell(30, 4, self.title)
        # Line break
        self.ln(20)

    # Page footer
    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        # helvetica italic 8
        self.set_font('Times', 'I', 8)
        # Page number
        self.cell(0, 10, 'Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')

def init_pdf(pdf: PDF, start_date: date = date(2021, 1,1), end_date: date = dt.now().date()):
    pdf.set_creation_date()
    pdf.set_author(author)
    pdf.set_creator(author)
    pdf.set_auto_page_break(auto = True, margin=20)
    pdf.set_section_title_styles(level0 = title_style_0, level1 = title_style_1,
                                level2 = title_style_2, level3 = title_style_3)
    pdf.set_title("Material Report, Endoscopy")
    pdf.set_font('Times', '', 12)
    pdf.add_page()
    pdf.start_section(f"Materialreport", level = 0)
    pdf.ln(15)
    pdf.cell(w = 100, txt = f"Universitätsklinikum Würzburg")
    pdf.ln(8)
    pdf.cell(w = 100, txt = f"Medizinische Klinik II")
    pdf.ln(8)
    pdf.cell(w = 100, txt = f"Gastroenterologische Endoskopie")
    pdf.ln(8)
    pdf.ln(8)
    pdf.cell(w = 100, txt = f"Von: {start_date.strftime(date_format)}")
    pdf.ln(8)
    pdf.cell(w = 100, txt = f"Bis: {end_date.strftime(date_format)}")
    pdf.add_page()
#     pdf.insert_toc_placeholder(render_toc, pages = 10)
#     pdf.add_page()
    

def add_summary(pdf, summary_dict, _type:str, cell_width: int = 100, cell_height: int = 0, spacing: int = 6):
    pdf.start_section(f"{_type}", level = 1)
    pdf.ln(spacing)

    for key, value in summary_dict.items():
        if _type == "Leistungen":
            try:
                pdf.start_section(f"{dgvs_lookup[key]} ({key})", level = 2)
            except:
                pdf.start_section(f"({key})", level = 2)
        else: 
            pdf.start_section(f"{key}", level = 2)
        pdf.ln(spacing)
        pdf.cell(w = cell_width, h = cell_height, txt = f"Anzahl:               {value['n_performed']}")
        pdf.ln(spacing)
        pdf.cell(w = cell_width, h = cell_height, txt = f"Mittlere Kosten:  {value['mean_cost']} Euro")
        pdf.ln(spacing)
        pdf.cell(w = cell_width, h = cell_height, txt = f"Mittlere Dauer:   {value['mean_duration']} min")
        pdf.ln(spacing)
        pdf.cell(w = cell_width, h = cell_height, txt = "Häufigste Materialien")
        pdf.ln(spacing)
        for material in value["top_most_used_mat_ids"]:
            pdf.cell(w = spacing)
            pdf.cell(w = cell_width, h = cell_height, txt = f"{material[3]}: Anzahl {material[1]}, Gesamtkosten: {material[2]} Euro")
            pdf.ln(spacing)
        pdf.ln(spacing)
        pdf.cell(w = cell_width, h = cell_height, txt = "Kumulativ Teuerste Materialien")
        pdf.ln(spacing)
        for material in value["top_highest_cost"]:
            pdf.cell(w = spacing)
            pdf.cell(w = cell_width, h = cell_height, txt = f"{material[3]}: Anzahl {material[1]}, Gesamtkosten: {material[2]} Euro")
            pdf.ln(spacing)
            
            
def add_material_info(pdf, mat_info, spacing:int = 6):
    pdf.add_page()
    pdf.start_section("Materialliste", level = 0)
    pdf.ln(spacing)
    for key, value in mat_info.items():
        pdf.ln(spacing)
        pdf.set_font('Times', 'B', 12)
        pdf.cell(w = 100, txt = value["name"])
        pdf.set_font('Times', '', 12)
        pdf.ln(spacing)
        pdf.cell(w = 30)
        pdf.cell(w = 70, txt = f'ID: {int(key)}')
        pdf.ln(spacing)
        pdf.cell(w=30)
        pdf.cell(w = 70, txt = f"Preis: {round(value['price'],2)} Euro")
        pdf.ln(spacing)
        pdf.cell(w=30)
        pdf.cell(w = 70, txt = f"Hersteller: {value['manufacturer']}")
        

In [51]:
# em.mat_info

In [56]:
pdf = PDF()
init_pdf(pdf, date(2021, 2,1), date(2021, 4,15))

In [57]:
add_summary(pdf, summary["intervention_types"], "Interventionstypen")
add_summary(pdf, summary["key_dgvs"], "Leistungen")
add_material_info(pdf, em.mat_info)

In [58]:
pdf.output('test.pdf')

In [55]:
summary["intervention_types"]

{'Endosonographie': {'n_performed': 135,
  'mean_cost': 36.35,
  'mean_duration': 16.39,
  'top_most_used_mat_ids': [(11125327.0,
    79,
    1824.11,
    'Ballon f.Ballonapplikator MAJ-249 steril'),
   (11126032.0, 58, 1351.4, 'Ballon f. Ballonapplikator  MAJ-233 ster'),
   (10168546.0, 17, 2002.77, 'Ultraschall-Kanüle HD-C G19'),
   (10168547.0, 8, 942.48, 'Ultraschall-Kanüle HD-C G22'),
   (11129089.0, 8, 54.24, '{Biopsiezange Ø4 240cm Bx40 steril einwe')],
  'top_highest_cost': [(10168546.0,
    17,
    2002.77,
    'Ultraschall-Kanüle HD-C G19'),
   (11125327.0, 79, 1824.11, 'Ballon f.Ballonapplikator MAJ-249 steril'),
   (11126032.0, 58, 1351.4, 'Ballon f. Ballonapplikator  MAJ-233 ster'),
   (11123722.0, 2, 1305.4, 'Gallengangstent Ø28/16/28mm gecovert'),
   (10168547.0, 8, 942.48, 'Ultraschall-Kanüle HD-C G22')]},
 'Gastroskopie': {'n_performed': 594,
  'mean_cost': 44.58,
  'mean_duration': 18.42,
  'top_most_used_mat_ids': [(11129089.0,
    275,
    1864.5,
    '{Biopsiezange

In [145]:
dt.now().date().strftime()

TypeError: strftime() missing required argument 'format' (pos 1)

In [21]:
# summary

In [8]:
report_summary = {
    "date_start": em.validation_report["mat_date_range"][0],
    "date_end": em.validation_report["mat_date_range"][1],
    "number_of_interventions_init": em.n_rows["mat_init"],
    "number_of_interventions_analyzed": em.n_rows["mat_after_processing"],
    "intervention_types": [(profession, (em.mat["intervention_type"]==profession).sum()) for profession in em.get_professions()]
}

TypeError: get_professions() missing 1 required positional argument: 'df'

In [10]:
profession = "Gastroskopie"

In [37]:
(em.mat["intervention_type"] == profession).sum()

594

In [34]:
def clean_dgvs_keys():
    replace = []
    keys = em.mat["key_dgvs"].to_list()
    for i, key in enumerate(keys):
        if "#" in key:
            elements = key.split("#")
            element_values = [[digit for digit in element if digit.isdigit()] for element in elements]
            element_values = [int("".join(digits)) for digits in element_values]
            index = element_values.index(max(element_values)) # if multiple numbers with same value exist, first is chosen
            replace.append((i, elements[index]))
            
    for replacement in replace:
        keys[replacement[0]] = replacement[1]
            
    return keys

In [20]:
profession_summary

{'Endosonographie': {'mean_cost': 36.35},
 'Gastroskopie': {'mean_cost': 44.58},
 'Koloskopie': {'mean_cost': 31.5},
 'ERCP': {'mean_cost': 80.25},
 'Ileokoloskopie': {'mean_cost': 6.23},
 'Enteroskopie/Single-,Doppelballon': {'mean_cost': 13.42},
 'Kapselendoskopie': {'mean_cost': 563.3},
 'Rektosigmoidoskopie': {'mean_cost': 246.0}}

In [ ]:
author = "T. J. Lux"

In [ ]:
title_style_0 = TitleStyle("Times", "B", 16)
title_style_1 = TitleStyle("Times", "B", 14)

In [ ]:
class PDF(FPDF):
    def header(self):
        # Logo
        self.image('data/logo.png', 8, 8, 15)
        # helvetica bold 15
        self.set_font('Times', '', 10)
        # Move to the right
        self.cell(20)
        # Title
        self.cell(30, 4, self.title)
        # Line break
        self.ln(20)

    # Page footer
    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        # helvetica italic 8
        self.set_font('Times', 'I', 8)
        # Page number
        self.cell(0, 10, 'Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')

In [ ]:
pdf = PDF(orientation="P", unit="mm", format="A4")

In [ ]:
pdf.set_section_title_styles(
    level0 = title_style_0,
    level1 = title_style_1
)

In [ ]:
pdf.set_creation_date()
pdf.set_author(author)
pdf.set_creator(author)
pdf.set_auto_page_break(auto = True, margin=2)
pdf.set_title("Material Report, Endoscopy")
pdf.set_font('Times', '', 16)




In [ ]:
pdf.add_page()
pdf.start_section("Summary", level = 0)
pdf.cell(40, 10, 'Hello World!')

In [ ]:
pdf.add_page()
pdf.start_section("Report by Intervention Room", level = 0)

In [ ]:
pdf.add_page()
pdf.start_section("Report by Intervention Type", level = 0)

In [ ]:
expected_mat_cols = ['Auftragsnummer', 'U-beginn Datum', 'U-Ende Datum', 'U-Dauer','Fachgebiet', 'Leistung_DGVS', 'Untersucher', 'Untersucher 2']
expected_imd_cols = ['Laufende Nr. der Materialdokumentation', 'Lfd.Nr', 'Angelegt am','Erfass.zeit', 'Verabreichungsdatum', 'Verabreichungszeit','Geändert am', 'Uhrzeit', 'Status Materialdokumentation',
    'Materialnummer', 'Materialtext', 'Name 1', 'Materialmenge','Mengeneinheit', 'Materialpreis', 'Währung', 'Anford. OE fachlich','Lfd. Nr. Leistung']

drop_cols_material = ["U-Ende Datum","Untersucher", "Untersucher 2"]
drop_cols_imd = ["Lfd.Nr", "Angelegt am", "Erfass.zeit", "Verabreichungszeit", "Geändert am", "Uhrzeit", "Anford. OE fachlich"]

rename_mat_cols = {
    "U-beginn Datum": "date",
    "Leistung_DGVS": "key_dgvs",
    "Fachgebiet": "intervention_type",
    "U-Dauer": "duration"
}

rename_imd_cols = {
    "Laufende Nr. der Materialdokumentation": "id_mat_doc",
    "Verabreichungsdatum": "date",
    "Materialnummer": "id_mat",
    "Materialtext": "name",
    "Name 1": "manufacturer",
    "Materialmenge": "quantity",
    "Mengeneinheit": "unit",
    "Währung": "currency"
}

In [ ]:
mat.rename(columns = rename_mat_cols)

In [39]:
mat = pd.read_excel(path_mat_doc)
imd = pd.read_excel(path_imd)

In [ ]:
mat.set_index("Auftragsnummer", inplace= True)

In [42]:
not_mat_cols = ["date", "duration", "intervention_type", "key_dgvs", "price_total"]

In [44]:
df = em.mat.drop(columns = not_mat_cols)
df.head()

,10129221.0,11126032.0,11125327.0,10168546.0,10132597.0,11101690.0,10196004.0,10186917.0,11101853.0,10125432.0,...,10174080.0,10172946.0,11100018.0,10191021.0,10148636.0,10119146.0,10178772.0,11122372.0,10164515.0,10157942.0
ID,,,,,,,,,,,,,,,,,,,,,
100006112,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100002310,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97671673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98573033,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99966521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def get_number(mat_id):
    select = em.imd["Materialnummer"] == mat_id
    tmp = em.imd[select]
    return tmp["Materialmenge"].sum()

for _ in most_expensive_mats[:10]:
#     print(_)
    print(f"{em.mat_info[_[0]]['text']}:")
    print(f"Preis pro Stück: {em.mat_info[_[0]]['price']}€")
    print(f"Verbraucht:{get_number(_[0])}\n")

In [ ]:
def get_time_series_summary(df):
    df = df.loc[:, ["U-beginn Datum", "U-Dauer", "Fachgebiet", "price_total"]]
    df = df[em.filter_time(df["U-beginn Datum"])]
    _dates = df["U-beginn Datum"].to_list()
    _professions = df["Fachgebiet"].to_list()
    _indexes = list(set(zip(_dates, _professions)))
    _dates, _professions = list(zip(*_indexes))
    
    df = df.groupby(["U-beginn Datum", "Fachgebiet"])
    summary = df.describe().T    

    time_series_dict = {
    "date": _dates,
    "profession": _professions, 
    "count": summary.loc[("U-Dauer", "count"),_indexes],
    "mean_duration": summary.loc[("U-Dauer", "mean"),_indexes],
    "mean_price": summary.loc[("price_total", "mean"),_indexes]
    }
    
    return pd.DataFrame(time_series_dict).sort_values(["date"])